In [3]:
from music21 import pitch
from pprint import pprint, pformat

import os
import random
import csv
import math
import time
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn.functional as F


from typing import List
from sklearn.model_selection import train_test_split

In [39]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

BLOCK_LENGTH = 11
FUTURE_LENGTH = 5
FINGER_SIZE = 5
BATCH_SIZE = 32
TRAIN_RATIO = 0.7
VAL_RATIO = 0.15
TEST_RATIO = 0.15
DATA_DIR = "/kaggle/input/pig-new"

cuda


In [6]:
block_future = [(11, 5)]
hands = ["right", "left"]
interval_to_midi = {
    # "Unison": 0,
    # "Minor Second": 1,
    # "Major Second": 2,
    # "Minor Third": 3,
    "Major Third": 4,
    # "Perfect Fourth": 5,
    # "Tritone": 6,
    # "Perfect Fifth": 7,
    # "Minor Sixth": 8,
    # "Major Sixth": 9,
    # "Minor Seventh": 10,
    # "Major Seventh": 11,
    # "Octave": 12
}


## Pitch Utilities


In [7]:
def extract_pitch_info(pitch):
    """
    Returns white key index and black key flag from pitch string.

    The white key index is centered around C4 (i.e., C4 → 0, C5 → 7, C3 → -7).

    Args:
        pitch (str): Note like "C4", "D#5", "Bb3".

    Returns:
        tuple: (white_key_val, black_key), where black_key is 1 for sharp/flat, 0 otherwise.

    """

    
    # Extract the base note and octave
    base_note = pitch[0]  # First character (e.g., "C", "D")
    octave = 4  # Default octave is 4 (octave start from middle C)
    
    note_val = {"C": 0, "D": 1, "E": 2, "F": 3, "G": 4, "A": 5, "B": 6}
    
    # Is the key a black key right next to the base note? (e.g., C#4, D#4)
    black_key = 0 # Default is white key

    # To tuple that split white/black keys, for allowing the black_key to reduce the span needed 
    # Note: Minus 4 to center around C4 (Middle c); Times 7 to span octaves (7 semitones)
    if pitch[1].isdigit(): # No sharp/flat like "C4"
        note_val[base_note] += (int(pitch[1]) - 4) * 7
    elif pitch[1] == "#": # Sharp(1 semitone up) like "C#4
        black_key = 1
        note_val[base_note] += (int(pitch[2]) - 4) * 7
    elif pitch[1] in ["b", "-"]: # Flat(1 semitone down) like "Cb4"
        black_key = 1

    return (note_val[base_note], black_key)

In [8]:
def get_white_black_diff(pitch1, pitch2):
    """
    Calculate the white-key distance between two pitches.
    
    Args:
        pitch1, pitch2: Note pitch in the format of "C4", "D#5", etc.

    Returns:
        int: Semitone distance between two pitches
    """
    
    a = extract_pitch_info(pitch1) 
    b =  extract_pitch_info(pitch2)
    return abs(a[0] - b[0]), a[1] + b[1] 

In [10]:
def pass_bounds(notes):
    """
    Checks if any note in the list is out of the allowed MIDI range.
    """
    
    pass_range = False
    for n in notes:
        if not (n == 0 or (21 <= n < 108)):
            pass_range = True
    return pass_range


def interval_symmetry(piece, interval):
    """
    Generates symmetrical piece by applying interval shifts across multiple octaves.
    
    Args:
        piece (PianoPiece): The original piano piece object.
        interval (int): The interval (in semitones) to use for symmetry transposition.

    Returns:
        list of PianoPiece: List of transposed, symmetrical versions of the input piece.
    """
    
    pieces = []
    octaves = [-9, -8, -7, -6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
    for octave in octaves:
        new_notes = [(n + (octave * interval)) if n != 0 else 0 for n in piece.notes]
        if not pass_bounds(new_notes):
            pieces.append(PianoPiece(new_notes, 
                                     piece.fingers, 
                                     piece.intervals, 
                                     piece.accidentals, 
                                     piece.ids, 
                                     
                                     piece.durations,
                                     piece.onset,
                                     piece.offset,

                                     piece.is_3_chord,
                                     piece.is_4_chord,
                                     piece.is_5_chord,
                                     piece.chord_tonic,
                                     piece.chord_sixth,
                                     piece.chord_second_inversion,
                                     
                                     piece.white_diff,
                                     piece.black_diff,
                                     piece.file_name))

    return pieces


## Chord Detection and Annotation

In [13]:
def detect_triad_type(notes):
    """
    Determines the type of chord based on a list of three note names.

    Args:
        notes (list of int): List of midi values.

    Returns:
        str: One of "tonic", "sixth", "second_inversion", or "unknown".
    """
    
    try:
        midi_notes = sorted([pitch.Pitch(n).midi for n in notes])
        intervals = [midi_notes[i] - midi_notes[0] for i in range(1, 3)]
        structure = (0, *intervals)

        if structure in [(0, 4, 7), (0, 3, 7)]:
            return "chord_tonic"
        elif structure in [(0, 3, 8), (0, 4, 9)]:
            return "chord_sixth"
        elif structure in [(0, 5, 9), (0, 5, 8)]:
            return "chord_second_inversion"
        else:
            return "unknown"
    except:
        return "unknown"


In [14]:
def mark_chords_with_type(df):
    """
    Adds one-hot encoded columns indicating chord type for triads.
    Also adds binary flags: is_3_chord, is_4_chord, is_5_chord.

    Args:
        df: DataFrame with a piano piece.

    Returns:
        Updated DataFrame with new one-hot encoded columns and chord size flags.
    """
    
    group_counts = df.groupby(["Onset", "Offset"]).size().reset_index(name="count")
    chord_groups = group_counts[group_counts["count"] >= 3][["Onset", "Offset", "count"]]

    df = df.merge(chord_groups.assign(is_chord=1), on=["Onset", "Offset"], how="left")
    df["is_chord"] = df["is_chord"].fillna(0).astype(int)
    df["count"] = df["count"].fillna(0).astype(int)

    df["chord_type"] = "none"

    for (onset, offset), group_df in df.groupby(["Onset", "Offset"]):
        if len(group_df) == 3:
            pitches = group_df["PitchName"].tolist()  
            chord_type = detect_triad_type(pitches)
            df.loc[(df["Onset"] == onset) & (df["Offset"] == offset), "chord_type"] = chord_type

    df["chord_type"] = df["chord_type"].replace("none", "unknown")

    dummies = pd.get_dummies(df["chord_type"], dtype='int')
    for i in dummies.columns:
        df[i] = dummies[i]

    df["is_3_chord"] = (df["count"] == 3).astype(int)
    df["is_4_chord"] = (df["count"] == 4).astype(int)
    df["is_5_chord"] = (df["count"] == 5).astype(int)
    
    # dropping unnecessary columns
    for i in ["count", "chord_type", "unknown", "is_chord"]:
        if i in df.columns:
            df = df.drop(i, axis=1)


    return df

In [15]:
def reorganize_fingers(df, hand="right"):
    """
    Reorganizes pitch data for chords or intervals.

    For each chord, the notes (and its rows) are sorted in ascending order.
    Example: Given a chord G4, E4, and C4 is reordered as C4, E4, G4.
    """

    df = df.copy()
    df["PitchPs"] = [pitch.Pitch(p).ps for p in df["PitchName"]]

    sorted_df = df.sort_values(by=["Onset", "PitchPs"])
    sorted_df["ID"] = range(len(sorted_df))
    sorted_df = sorted_df.drop(columns=["PitchPs"])
    
    return sorted_df


## Modeling and Data Structures

In [17]:
class PianoPiece:
    """
    A data structure representing a piano piece with associated musical and fingering information.

    Attributes:
        notes : MIDI values representing the notes in the piece.
        fingers : Finger numbers corresponding to each note.
        intervals : Interval distances between consecutive notes.
        accidentals : Accidentals (sharps, flats, naturals) for each note.
        ids : Unique identifiers for each note or event.
        durations : Durations for each note in seconds.
        onset : Onset times (in seconds) for each note.
        offset : Offset times (in seconds) for each note.

        white_diff : Movement in terms of white keys between notes.
        black_diff : Movement in terms of black keys between notes.

        chords : Whether the note belongs to a chord.
        chord_tonic : Whether the note is in the root (tonic) chord.
        chord_sixth : Whether the note is in a sixth chord.
        chord_second_inversion : Whether the note is in a second inversion chord.

        file_name : Name of the file from which the data was derived.
    """

    def __init__(
        self,
        notes = None, fingers = None, intervals = None,
        accidentals = None, ids = None,
        durations = None, onset = None, offset = None,
        is_3_chord = None, is_4_chord = None, is_5_chord = None,
        chord_tonic = None, chord_sixth = None, chord_second_inversion = None,
        white_diff = None, black_diff = None,
        file_name: str = ""
    ):
        self.notes = notes
        self.fingers = fingers
        self.intervals = intervals
        self.accidentals = accidentals
        self.ids = ids
        self.durations = durations
        self.onset = onset
        self.offset = offset

        self.white_diff = white_diff
        self.black_diff = black_diff

        self.is_3_chord = is_3_chord
        self.is_4_chord = is_4_chord
        self.is_5_chord = is_5_chord
        self.chord_tonic = chord_tonic
        self.chord_sixth = chord_sixth
        self.chord_second_inversion = chord_second_inversion

        self.file_name = file_name

    def get_features(self, input_features):
        """Returns selected features in aligned format for model input."""
    
        feature_map = {
            "fingers": self.fingers[:-1],
            "notes": self.notes[:-1],
            "intervals": self.intervals,
            
            "accidentals_current": self.accidentals[:-1],
            "accidentals_next": self.accidentals[1:],
            "white_diff": self.white_diff[:-1],
            "black_diff": self.black_diff[:-1],
            # "chords": self.chords[:-1],
            
            "is_3_chord": self.is_3_chord[:-1],
            "is_4_chord": self.is_4_chord[:-1],
            "is_5_chord": self.is_5_chord[:-1],
            "chord_tonic": self.chord_tonic[:-1],
            "chord_sixth": self.chord_sixth[:-1],
            "chord_second_inversion": self.chord_second_inversion[:-1],
        }
        
        # Detect invalid features
        invalid_features = [name for name in input_features if name not in feature_map]
        if invalid_features:
            raise ValueError(f"The following input features are invalid: {invalid_features}")
    
        selected = [feature_map[name] for name in input_features]
    
        if not selected:
            raise ValueError("No valid features provided for model input.")
    
        return [list(x) for x in zip(*selected)]


In [19]:
def check_finger_ranges(df, hand, file_path):
    """
    Validates that finger values in the DataFrame fall within the correct range 
    for the specified hand. 

    Args:
        df: DataFrame with a piano piece
        hand (str): Either 'right' or 'left', indicating the hand being checked.
    """
    
    df["Finger"] = df["Finger"].astype(str).str.split('_').str[0].astype(int)

    if hand == "right":
        invalid_rows = df[(df["Finger"] < 1) | (df["Finger"] > 5)]
        if not invalid_rows.empty:
            raise ValueError(f"Invalid right-hand finger values found in {file_path}:\n{invalid_rows}")

    elif hand == "left":
        invalid_rows = df[(df["Finger"] < -5) | (df["Finger"] > -1)]
        if not invalid_rows.empty:
            raise ValueError(f"Invalid left-hand finger values found in {file_path}:\n{invalid_rows}")

    else:
        raise ValueError(f"Invalid hand type: {hand}. Expected 'right' or 'left'.")




In [20]:
def load_piano_piece(file_path, hand="right", use_white_black=False, aug=False):
    """
    Loads data from a file, performs feature engineering
    and optionally applies data augmentation using interval transpositions.

    Returns:
        list of PianoPiece objects
    """
    
    df = pd.read_csv(file_path, header=None)
    df.columns = ["ID", "Onset", "Offset", "PitchName", "Column4", "Column5", "Beam", "Finger"]
    
    if df.empty:
        print(f"No data for {filepath}")
        return []
    df = df.drop(columns=["Column4", "Column5"])
    
    df["Onset"] = df["Onset"].astype(float)
    df["Offset"] = df["Offset"].astype(float)
    
    # Select hand
    df = df[df["Beam"] == (0 if hand == "right" else 1)]
    df = df.reset_index(drop=True)
    df["Finger"] = df["Finger"].astype(str).str.split('_').str[0].astype(int)
    df = reorganize_fingers(df, hand=hand)
    check_finger_ranges(df, hand, file_path)
    
    # numeric pitch and accidental flag
    df["Note"] = df["PitchName"].apply(lambda x: pitch.Pitch(x).ps)
    df["Accidental"] = df["PitchName"].apply(lambda x: int(pitch.Pitch(x).accidental is None))

    # prepare diff columns
    df["white_diff"] = 0
    df["black_diff"] = 0
    
    if use_white_black:
        for i in range(1, len(df)):
            w, b = get_white_black_diff(df.loc[i-1,"PitchName"], df.loc[i,"PitchName"])
            df.loc[i, "white_diff"] = w
            df.loc[i, "black_diff"] = b

    chord_labels = [
        "is_3_chord", "chord_tonic", "chord_sixth", "chord_second_inversion",
        "is_4_chord", "is_5_chord",
    ]
    for lbl in chord_labels:
        df[lbl] = 0
    df = mark_chords_with_type(df)

    df["Duration"] = (df["Offset"] - df["Onset"]).round(2)
    
    notes = df["Note"].tolist()
    fingers = df["Finger"].tolist()
    white_diff = df["white_diff"].tolist()
    black_diff = df["black_diff"].tolist()
    accidentals = df["Accidental"].tolist()
    ids = df["ID"].astype(int).tolist()
    onset = df["Onset"].round(2).tolist()
    offset = df["Offset"].round(2).tolist()
    
    is_3_chord = df["is_3_chord"].tolist()
    is_4_chord = df["is_4_chord"].tolist()
    is_5_chord = df["is_5_chord"].tolist()
    chord_tonic = df["chord_tonic"].tolist()
    chord_sixth = df["chord_sixth"].tolist()
    chord_second_inversion = df["chord_second_inversion"].tolist()
    
    intervals = np.diff(np.array(notes, dtype=int)).tolist()

    # normalize fingers for loss calculation
    if hand == "right":
        fingers = [f - 1 for f in fingers]
    else:
        fingers = [-f - 1 for f in fingers]

    piece = PianoPiece(
        notes=notes,
        fingers=fingers,
        intervals=intervals,
        accidentals=accidentals,
        ids=ids,
        # durations=durations,
        onset=onset,
        offset=offset,
        white_diff=white_diff,
        black_diff=black_diff,
        
        is_3_chord=is_3_chord,
        is_4_chord=is_4_chord,
        is_5_chord=is_5_chord,
        chord_tonic=chord_tonic,
        chord_sixth=chord_sixth,
        chord_second_inversion=chord_second_inversion,
        file_name=file_path
    )

    if aug:
        pieces = []
        for interval in interval_to_midi.values():
            pieces.extend(interval_symmetry(piece, interval))
        return pieces
    else:
        return [piece]


In [21]:
def slide_window_future_gen(input_list, window_size, future_size):
    """
    Generates sliding windows of fixed size from the input list, with the future part of the window set to zero.
    
    Args:
        input_list (list): A list of input data (e.g., finger positions, intervals).
        window_size (int): The length of the sliding window.
        future_size (int): The number of future time steps to mask (set to zero).
        
    Yields:
        list: A sliding window of size `window_size`, with the last `future_size` elements set to zero.
    """
    
    for start in range(len(input_list) - window_size + 1):
        full_list = input_list[start : start + window_size]
        
        for i in range(window_size-future_size, window_size):
            full_list[i][0] = 0
        
        yield full_list


def prepare_inputs(file_paths, hand="right",  input_features=None, aug=False):
    """
    Prepare inputs for the neural network.

    Returns:
        tuple: 
            - inputs (list): A list of input sequences for the neural network.
            - labels (list): A list of target labels (finger positions) corresponding to the input sequences.
            - processed_data (dict): A dictionary mapping each filename to the number of pieces processed.
    """

    inputs = []
    labels = []
    processed_data = {}
    vector_list = []

    if input_features is None:
        input_features = ["fingers", "intervals", "accidentals_current", "accidentals_next"]
        
    # for filename in tqdm(sorted(filenames), desc="Processing Files"):
    for filename in sorted(file_paths):
        vector_list = []
        
        if "white_diff" in input_features or "black_diff" in input_features:
            pieces = load_piano_piece(filename, hand, use_white_black=True, aug=aug)
        else:
            pieces = load_piano_piece(filename, hand, aug=aug)
            
        for piece in pieces:
            feature_matrix = piece.get_features(input_features)
            vector_list.append(feature_matrix)
            
        processed_data[filename] = len(pieces)
        
        for i in range(len(vector_list)):
            inputs.extend(
                [l for l in slide_window_future_gen(vector_list[i], BLOCK_LENGTH, FUTURE_LENGTH)]
            )
            labels.extend(
                [f for f in pieces[i].fingers[BLOCK_LENGTH - FUTURE_LENGTH : -FUTURE_LENGTH]]
            )

        
    return inputs, labels, processed_data


### Loading and splitting data 

In [29]:
class PianoFingeringDataset(Dataset):
    """
    A dataset class for loading piano fingering data, designed for use with PyTorch DataLoader.
    """
    
    def __init__(self, file_paths, hand="right", input_features=None, aug=False):
        self.input_list, self.label_list, self.processed_data = prepare_inputs(file_paths, hand, input_features, aug)

    def __len__(self):
        return len(self.input_list)

    def __getitem__(self, idx):
        
        x = torch.tensor(self.input_list[idx], dtype=torch.float32)
        y = torch.tensor(self.label_list[idx], dtype=torch.int64)  
        return x, y

In [22]:
def split_files(files, train_ratio, val_ratio, test_ratio):
    train_files, temp_files = train_test_split(files, 
                                               test_size=(val_ratio + test_ratio), 
                                               random_state=48)
    val_files, test_files = train_test_split(temp_files, 
                                             test_size=(test_ratio / (val_ratio + test_ratio)), 
                                             random_state=48)

    return train_files, val_files, test_files
    

In [23]:
def prepare_data(data_dir, batch_size, train_ratio, val_ratio, test_ratio, hand="right", input_features=None, aug=False):
    """
    Prepares the data by splitting it into train, validation, and test sets, then creates DataLoader objects.
    """

    all_files = os.listdir(data_dir)

    full_paths = [os.path.join(data_dir, f) for f in all_files]
    
    print(f"Working with {hand} hand!")
    train_files, val_files, test_files = split_files(full_paths, train_ratio, val_ratio, test_ratio)

    print(f"Train set {len(train_files)}")
    print(f"Vaidation set {len(val_files)}")
    print(f"Test set {len(test_files)}\n")
    
    train_dataset = PianoFingeringDataset(train_files, hand, input_features, aug)
    val_dataset = PianoFingeringDataset(val_files, hand, input_features, aug)
    test_dataset = PianoFingeringDataset(test_files, hand, input_features)

    if aug:
        len_train = 0
        for i in train_dataset.processed_data.values():
            len_train += int(i)
        print(f"Train set after aug {len_train}")
        
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

    return train_loader, val_loader, test_loader, train_files, val_files, test_files

## Models

In [24]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, block_length, future_length, num_layers=1):
        super(LSTM, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.block_length = block_length
        self.future_length = future_length
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size, hidden_size, bidirectional=True, batch_first=True, num_layers=num_layers)
        self.lambda_layer_idx = block_length - future_length - 1
        self.dropout = nn.Dropout(p=0.4)
        self.fc = nn.Linear(hidden_size * 2, output_size)  
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)  
        selected_output = lstm_out[:, self.lambda_layer_idx, :]  
        logits = self.fc(selected_output)  
        probabilities = self.softmax(logits)
        return probabilities

    def __str__(self):
        return f"LSTM(h={self.hidden_size}, layers={self.num_layers}, block={self.block_length}, future={self.future_length})"

In [25]:
class GRU(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, block_length, future_length, num_layers=1):
        super(GRU, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.block_length = block_length
        self.future_length = future_length
        self.num_layers = num_layers
        
        self.gru = nn.GRU(input_size, hidden_size, bidirectional=True, batch_first=True, num_layers=num_layers)
        self.lambda_layer_idx = block_length - future_length - 1
        self.dropout = nn.Dropout(p=0.4)
        self.fc = nn.Linear(hidden_size * 2, output_size)  
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        gru_out, _ = self.gru(x)  
        selected_output = gru_out[:, self.lambda_layer_idx, :]  
        logits = self.fc(selected_output)  
        probabilities = self.softmax(logits)
        return probabilities

    def __str__(self):
        return f"GRU(h={self.hidden_size}, layers={self.num_layers}, block={self.block_length}, future={self.future_length})"

## Training

In [36]:
def train_model(model, device, train_loader, val_loader, num_epochs,
                lr=0.001, weights=None, name=None, log_file=None):
    
    model = model.to(device)

    criterion = nn.CrossEntropyLoss(weight=weights.to(device) if weights is not None else None)

    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)

    best_val_accuracy = 0.0
    best_model_state = None

    # List to store logs for each epoch
    log_data = []

    for epoch in range(num_epochs):
        start_time = time.time()

        # Training phase
        model.train()
        epoch_loss = 0.0
        correct = 0
        total = 0

        for idx, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        epoch_loss /= total
        train_accuracy = correct / total

        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for idx, (inputs, labels) in enumerate(val_loader):
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                val_correct += (predicted == labels).sum().item()
                val_total += labels.size(0)

        val_loss /= val_total
        val_accuracy = val_correct / val_total

        epoch_time = time.time() - start_time

        # Print to console
        # if (epoch + 1) % 5 == 0:
        print(f"Epoch [{epoch + 1}/{num_epochs}], "
              f"Train Loss: {epoch_loss:.4f}, "
              f"Train Accuracy: {train_accuracy:.4f}, "
              f"Time: {epoch_time:.2f} sec ")
        print(f"Epoch [{epoch + 1}/{num_epochs}], "
              f"Val Loss: {val_loss:.4f}, "
              f"Validation Accuracy: {val_accuracy:.4f}\n")

        # Store the data for logging
        log_data.append({
            'Epoch': epoch + 1,
            'Train Loss': round(epoch_loss, 3),
            'Train Accuracy': round(train_accuracy, 3),
            'Val Loss': round(val_loss, 3),
            'Val Accuracy': round(val_accuracy, 3)
        })

        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_model_state = model.state_dict().copy()

    if name and best_model_state:
        print(f"Saving best model with validation accuracy {best_val_accuracy:.4f} as {name}")
        torch.save({
            'model_state_dict': best_model_state,
            'model_config': {
                'input_size': model.input_size,
                'hidden_size': model.hidden_size,
                'output_size': model.fc.out_features,
                'block_length': model.block_length,
                'future_length': model.future_length,
                'num_layers': model.num_layers
            }
        }, name)

    if best_model_state:
        model.load_state_dict(best_model_state)

    # Convert the log data to a DataFrame and save as CSV
    # log_df = pd.DataFrame(log_data)
    # log_df.to_csv(log_file, index=False)
    # print(f"Training logs saved to {log_file}")

    return model


In [30]:
features_by_hand = {
    # lstm best features
    # "right":  ['fingers', 'intervals',  'chord_tonic', 'chord_sixth', 'chord_second_inversion'],
    # "left":  ['fingers', 'intervals',  'chord_tonic', 'chord_sixth', 'chord_second_inversion'],

    # gru best features
    "right":  ['fingers', 'intervals', "accidentals_current", "accidentals_next", 'chord_tonic', 'chord_sixth', 'chord_second_inversion'],
    "left":  ['fingers','intervals', "accidentals_current", "accidentals_next",  'chord_tonic', 'chord_sixth', 'chord_second_inversion'],
}

DATA_DIR = "/kaggle/input/pig-new"
# DATA_DIR = "/kaggle/input/pig-own-data"
# DATA_DIR = "/kaggle/input/own-piano-data"

data_loaders = {}
for hand in ["right", "left"]:
    features = features_by_hand[hand]

    train_loader, val_loader, test_loader, \
    train_files, val_files, test_files = prepare_data(
        DATA_DIR, BATCH_SIZE, TRAIN_RATIO, VAL_RATIO, TEST_RATIO,
        hand=hand, aug=False, input_features=features
    )

    data_loaders[hand] = {
        "train": train_loader,
        "val": val_loader,
        "test": test_loader,
        "train_files": train_files,
        "val_files": val_files,
        "test_files": test_files
    }



Working with right hand!
Train set 216
Vaidation set 46
Test set 47

Working with left hand!
Train set 216
Vaidation set 46
Test set 47



In [65]:
# the best parameters were obtained as a result of experiments
hyperparams_by_model = {
    "GRU": {
        "hidden_size": 16,
        "num_layers": 3,
    },
    "LSTM": {
        "hidden_size": 64,
        "num_layers": 2,
    },
}

models = {
    "right": None,
    "left": None,
}


for hand in hands:
    hparams = hyperparams_by_model["GRU"]
    print(f"Training {hand} hand ")
    
    model = GRU(input_size=len(features_by_hand[hand]),
                hidden_size=hparams['hidden_size'],
                output_size=FINGER_SIZE,
                block_length=BLOCK_LENGTH,
                future_length=FUTURE_LENGTH,
                num_layers=hparams['num_layers'])
    
    print(f"INFO: {model}")
    
    loaders = data_loaders[hand]
    train_model(model, device, loaders["train"], loaders["val"], 20)
    models[hand] = model
    
    print("__________________________________________________________\n")


Training right hand 
INFO: GRU(h=16, layers=3, block=11, future=5)
Epoch [1/20], Train Loss: 1.3126, Train Accuracy: 0.5910, Time: 5.18 sec 
Epoch [1/20], Val Loss: 1.2539, Validation Accuracy: 0.6495

Epoch [2/20], Train Loss: 1.2287, Train Accuracy: 0.6756, Time: 5.14 sec 
Epoch [2/20], Val Loss: 1.2416, Validation Accuracy: 0.6613

Epoch [3/20], Train Loss: 1.2149, Train Accuracy: 0.6875, Time: 5.11 sec 
Epoch [3/20], Val Loss: 1.2359, Validation Accuracy: 0.6638

Epoch [4/20], Train Loss: 1.2062, Train Accuracy: 0.6965, Time: 5.15 sec 
Epoch [4/20], Val Loss: 1.2448, Validation Accuracy: 0.6543

Epoch [5/20], Train Loss: 1.1998, Train Accuracy: 0.7015, Time: 5.11 sec 
Epoch [5/20], Val Loss: 1.2293, Validation Accuracy: 0.6709

Epoch [6/20], Train Loss: 1.1962, Train Accuracy: 0.7054, Time: 5.49 sec 
Epoch [6/20], Val Loss: 1.2344, Validation Accuracy: 0.6661

Epoch [7/20], Train Loss: 1.1897, Train Accuracy: 0.7117, Time: 5.13 sec 
Epoch [7/20], Val Loss: 1.2287, Validation Accura

## Evaluating

In [50]:
def update_state_with_prediction(old_state, finger_pred, new_vec, future_size):
    pred = old_state[-future_size]
    pred[0] = finger_pred  # Update the predicted finger

    # Updating the state with the new vector as a tensor
    new_state = torch.tensor([0] + new_vec, dtype=torch.float32)
    old_state[-future_size] = pred
    return old_state[1:] + [new_state]  


In [63]:
def prepare_test_inputs(file_path, hand="right", input_features=None):
    inputs = []
    labels = []
    if "white_diff" in input_features or "black_diff" in input_features:
        pieces = load_piano_piece(file_path, hand, use_white_black=True)
    else:
        pieces = load_piano_piece(file_path, hand)
        
    feature_matrix = pieces[0].get_features(input_features)
        
    inputs.append(feature_matrix)
    labels.append(pieces[0].fingers)
    
    return inputs, labels, pieces[0].ids

In [59]:
def predict_fingerings(input_list, label_list, model):
    model.eval()
    results = []

    with torch.no_grad():
        for test_vector, test_finger in zip(input_list, label_list):
            init_state_b = [
                torch.tensor([test_finger[i]] + test_vector[i], dtype=torch.float32)
                for i in range(model.block_length - model.future_length)
            ]
            init_state_a = [
                torch.tensor([0] + test_vector[i], dtype=torch.float32)
                for i in range(model.block_length - model.future_length, model.block_length)
            ]

            init_state = init_state_b + init_state_a
            num_intervals = len(test_vector)
            temp_finger_res = []

            for test_step in range(0, num_intervals - model.block_length + 1):
                np_init_state = (
                    torch.stack(init_state)
                    .view(-1, model.block_length, model.input_size)
                    .to(device)
                )
                
                pred_prob = model(np_init_state)
                finger_pred = torch.argmax(pred_prob, dim=1).item()
                temp_finger_res.append(finger_pred)

                if test_step < num_intervals - model.block_length - 1:
                    next_vector = test_vector[test_step + model.block_length]
                    init_state = update_state_with_prediction(
                        init_state, finger_pred, next_vector, model.future_length
                    )

            temp_finger_res = (test_finger[: model.block_length - model.future_length] + temp_finger_res + test_finger[-model.future_length:])
            results.append(temp_finger_res)

    return results


In [60]:
def evaluate_fingering(test_files, hand, model, input_features=None):
    total_correct = 0
    total_predictions = 0

    for file in test_files:
        
        test_input_list, test_label_list, test_id_list = prepare_test_inputs(file, hand, input_features)
        predicted_fingerings = predict_fingerings(test_input_list, test_label_list, model)
        
        flat_pred = [pred for pred in predicted_fingerings[0]]
        # transformation = {4: 0, 0: 4, 3: 1, 1: 3}

        # flat_pred = [transformation.get(pred, pred) for pred in flat_pred]

        # print(flat_pred)
        flat_label = [gt for gt in test_label_list[0]]
        correct = sum(p == gt for p, gt in zip(flat_pred, flat_label))
        total_correct += correct
        total_predictions += len(flat_label)
        
        file_accuracy = correct / len(flat_label) if len(flat_label) > 0 else 0
        print(f"File: {file} | Accuracy: {file_accuracy:.4f}")

    overall_accuracy = total_correct / total_predictions if total_predictions > 0 else 0
    print(f"Overall Accuracy: {overall_accuracy:.4f}\n")
    return overall_accuracy


In [66]:
for hand in ["right", "left"]:
    print(f"Evaluating {hand} hand")
    print(model)
    
    # excluding fingers feature because of evaluating
    feature = features_by_hand[hand][1:]
    
    evaluate_fingering(test_files[:], hand, models[hand], feature)

Evaluating right hand
GRU(h=16, layers=3, block=11, future=5)
File: /kaggle/input/pig-new/040-1_fingering.csv | Accuracy: 0.6667
File: /kaggle/input/pig-new/026-5_fingering.csv | Accuracy: 0.8576
File: /kaggle/input/pig-new/051-1_fingering.csv | Accuracy: 0.6359
File: /kaggle/input/pig-new/030-4_fingering.csv | Accuracy: 0.8889
File: /kaggle/input/pig-new/126-1_fingering.csv | Accuracy: 0.7442
File: /kaggle/input/pig-new/022-3_fingering.csv | Accuracy: 0.8477
File: /kaggle/input/pig-new/041-1_fingering.csv | Accuracy: 0.6389
File: /kaggle/input/pig-new/003-5_fingering.csv | Accuracy: 0.6041
File: /kaggle/input/pig-new/007-1_fingering.csv | Accuracy: 0.6703
File: /kaggle/input/pig-new/047-2_fingering.csv | Accuracy: 0.7135
File: /kaggle/input/pig-new/015-3_fingering.csv | Accuracy: 0.6536
File: /kaggle/input/pig-new/097-1_fingering.csv | Accuracy: 0.6741
File: /kaggle/input/pig-new/038-1_fingering.csv | Accuracy: 0.8158
File: /kaggle/input/pig-new/019-3_fingering.csv | Accuracy: 0.6870
